In [39]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec

Data Importation

In [15]:
df = pd.read_json("./data/Cell_Phones_and_Accessories_5.json", lines=True)
df.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
5,APX47D16JOP7H,120401325X,RLH,"[1, 2]",These make using the home button easy. My daug...,3,Cute,1381536000,"10 12, 2013"
6,A1JVVYYO7G56DS,120401325X,Tyler Evans,"[0, 0]",Came just as described.. It doesn't come unstu...,5,best thing ever..,1377129600,"08 22, 2013"
7,A6FGO4TBZ3QFZ,3998899561,Abdullah Albyati,"[1, 2]",it worked for the first week then it only char...,1,not a good Idea,1384992000,"11 21, 2013"
8,A2JWEDW5FSVB0F,3998899561,Adam,"[2, 3]","Good case, solid build. Protects phone all aro...",5,Solid Case,1380067200,"09 25, 2013"
9,A8AJS1DW7L3JJ,3998899561,Agata Majchrzak,"[1, 1]",This is a fantastic case. Very stylish and pro...,5,Perfect Case,1396483200,"04 3, 2014"


In [7]:
df.tail(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
194429,AF30DHAVSYNZ0,B00LH1R6SK,Sasha DLH,"[0, 0]","Ahah, I love it. The case looks better than th...",5,Yesssssss!,1405641600,"07 18, 2014"
194430,A34KTJ4VWNBOHI,B00LH1R6SK,Stina,"[0, 0]","As someone who owns an iPhone, I absolutely lo...",5,Beautiful Color and Texture,1405641600,"07 18, 2014"
194431,A1Q1Z7IR79G52A,B00LORXVUE,"Ann Ingersoll Madison Rain ""Ann""","[0, 0]",What a high quality cable. Right out of the pa...,5,"Must have, high quality charging cable for ipa...",1405987200,"07 22, 2014"
194432,A388OX0ZF7OT8R,B00LORXVUE,Chuck.C,"[0, 0]",This one actually works unlike some others I b...,4,Long Wire,1405728000,"07 19, 2014"
194433,A2BLFCOPSMBOZ9,B00LORXVUE,Dave Edmiston,"[0, 0]","I don't like the word ""hate"", so let's just sa...",5,Awesome cable!,1405641600,"07 18, 2014"
194434,A1YMNTFLNDYQ1F,B00LORXVUE,eyeused2loveher,"[0, 0]",Works great just like my original one. I reall...,5,This works just perfect!,1405900800,"07 21, 2014"
194435,A15TX8B2L8B20S,B00LORXVUE,Jon Davidson,"[0, 0]",Great product. Great packaging. High quality a...,5,Great replacement cable. Apple certified,1405900800,"07 21, 2014"
194436,A3JI7QRZO1QG8X,B00LORXVUE,Joyce M. Davidson,"[0, 0]","This is a great cable, just as good as the mor...",5,Real quality,1405900800,"07 21, 2014"
194437,A1NHB2VC68YQNM,B00LORXVUE,Nurse Farrugia,"[0, 0]",I really like it becasue it works well with my...,5,I really like it becasue it works well with my...,1405814400,"07 20, 2014"
194438,A1AG6U022WHXBF,B00LORXVUE,Trisha Crocker,"[0, 0]","product as described, I have wasted a lot of m...",5,I have wasted a lot of money on cords,1405900800,"07 21, 2014"


To build an sentiments analyses model, w'are intersting just in the reviewText and overall columns, so we extract just the two columns and drop others columns

In [16]:
# Drop unnecessary columns
columns_to_drop = ['reviewerID', 'asin', 'reviewerName', 'helpful', 'summary', 'unixReviewTime', 'reviewTime']
df = df.drop(columns_to_drop, axis=1)

In [17]:
# Verify the updated DataFrame
df.head

<bound method NDFrame.head of                                                reviewText  overall
0       They look good and stick good! I just don't li...        4
1       These stickers work like the review says they ...        5
2       These are awesome and make my phone look so st...        5
3       Item arrived in great time and was in perfect ...        4
4       awesome! stays on, and looks great. can be use...        5
...                                                   ...      ...
194434  Works great just like my original one. I reall...        5
194435  Great product. Great packaging. High quality a...        5
194436  This is a great cable, just as good as the mor...        5
194437  I really like it becasue it works well with my...        5
194438  product as described, I have wasted a lot of m...        5

[194439 rows x 2 columns]>

In [18]:
#Verify messing value
missing_values_count = df.isnull().sum()
print(missing_values_count)

reviewText    0
overall       0
dtype: int64


NB: wa observe that, when the over all is less or equal 2 the review is negative, and when it heigher then 2 the review is positive, so after this step we encode the overall column to 1-->positve , 0 ---> negative

In [22]:
filtered_df = df.loc[df['overall'] <=2, 'reviewText']
print(filtered_df)

7         it worked for the first week then it only char...
19        It worked great for the first couple of weeks ...
26        I am disappointed that the 1A didn't work with...
28                         After a week only one side works
32        Only works one side at a time. When you connec...
                                ...                        
194299    I would not recommend this to anyone. The mate...
194302    I used to have this case for my ipods and they...
194304    I tried to like this product, I got a Warrenty...
194309    This is okay if you don't ever need to access ...
194346                              Not OEM, cheap knockoff
Name: reviewText, Length: 24343, dtype: object


In [23]:
filtered_df = df.loc[df['overall'] >2, 'reviewText']
print(filtered_df)

0         They look good and stick good! I just don't li...
1         These stickers work like the review says they ...
2         These are awesome and make my phone look so st...
3         Item arrived in great time and was in perfect ...
4         awesome! stays on, and looks great. can be use...
                                ...                        
194434    Works great just like my original one. I reall...
194435    Great product. Great packaging. High quality a...
194436    This is a great cable, just as good as the mor...
194437    I really like it becasue it works well with my...
194438    product as described, I have wasted a lot of m...
Name: reviewText, Length: 170096, dtype: object


In [24]:
#Transformation de la variables cibles en une variable numérique binaire
# 0: negative ; 1: positive
df['overall'] = df['overall'].apply(lambda val: 0 if val <=2 else 1)

In [25]:
df['overall'].value_counts(normalize=True)

1    0.874804
0    0.125196
Name: overall, dtype: float64

Split the data to tarin and test and validate

In [35]:
seed = 123

train, test = train_test_split(df, test_size=0.3, random_state=seed, stratify=df['overall'])
test, validate = train_test_split(test, test_size=0.5, random_state=seed, stratify=test['overall'])

print("train data: ", train.shape)
print("Validation data: ", validate.shape)
print("test data: ", test.shape)

train data:  (136107, 2)
Validation data:  (29166, 2)
test data:  (29166, 2)


In [36]:
#Save data
train.to_csv('./data/train.csv', index=False)
validate.to_csv('./data/validate.csv', index=False)
test.to_csv('./data/test.csv', index=False)

Data Processing

In [41]:
def preprocess_text(text):

    # Remove HTML tags
    text = re.sub('<[^>]*>', '', text)

    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub('[^a-zA-Z]', ' ', text).lower()

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Combine words back into a single string
    preprocessed_text = ' '.join(words)

    return preprocessed_text

In [42]:
train['preprocessed_text']= train['reviewText'].apply(preprocess_text)
test['preprocessed_text']= test['reviewText'].apply(preprocess_text)
validate['preprocessed_text']= validate['reviewText'].apply(preprocess_text)

In [43]:
train.head()

,reviewText,overall,preprocessed_text
42227,Looks just like the original HTC cable and wor...,1,look like original htc cable work perfect boug...
124534,"It didn't work for me, my outlets wouldn't kee...",0,work outlet keep charger straight connect well...
72556,This was one of my 1st cases for my phone. I l...,1,one st case phone love others recommend anyone...
40622,"really cool, very efficent",1,really cool efficent
183403,If you really want to protect your iPhone this...,1,really want protect iphone case completely enc...


Vectorieation using BOW, TF-IDF, Word2vec

In [44]:
def train_svm_with_representations(data_train, data_test, representation):
    if representation == 'bow':
        vectorizer = CountVectorizer()
    elif representation == 'tfidf':
        vectorizer = TfidfVectorizer()
    else:
        raise ValueError("Invalid representation. Choose 'bow' or 'tfidf'.")

    X_train = vectorizer.fit_transform(data_train)
    X_test = vectorizer.transform(data_test)

    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return y_pred

In [45]:
def get_word2vec_embeddings(data):
    tokenized_sentences = [sentence.split() for sentence in data]
    model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)
    
    # Initialize an empty array to store the embeddings
    embeddings = np.zeros((len(tokenized_sentences), model.vector_size))
    
    # Populate the embeddings array with word vectors
    for i, sentence in enumerate(tokenized_sentences):
        word_vectors = [model.wv[word] for word in sentence if word in model.wv]
        if word_vectors:
            embeddings[i] = np.mean(word_vectors, axis=0)
    
    return embeddings

def train_svm_with_word2vec(data_train, data_test, y_train):
    X_train = get_word2vec_embeddings(data_train)
    X_test = get_word2vec_embeddings(data_test)
    
    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return y_pred


In [47]:
X_train = train['preprocessed_text']
y_train = train['overall']
X_test = test['preprocessed_text']
y_test = test['overall']

In [ ]:
# TF-IDF
y_pred_tfidf = train_svm_with_representations(X_train, X_test, 'tfidf')
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print(accuracy_tfidf)

In [ ]:
# Word2Vec
y_pred_word2vec = train_svm_with_word2vec(X_train, X_test, y_train)
accuracy_word2vec = accuracy_score(y_test, y_pred_word2vec)
print(accuracy_word2vec)